In this project, we will explore and analyze data from NYC Taxi and Limousine Commission (TLC) in June 2018. We then build a predictive model for whether a trip will be tipped or not, which is a classification problem. By interpreting the feature importances, we will see what types of trips are more likely to be tipped. 

Data is downloaded from https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page (2018 June Green)

The green taxi trip records include fields capturing pick-up and drop-off dates/times, pick-up and drop-off locations, trip distances, itemized fares, rate types, payment types, and driver-reported passenger counts. 


# Import Packages

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None) # this will show all the columns in dataframe

import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline 

from datetime import datetime

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix, roc_auc_score, r2_score, mean_squared_error

import time

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

import warnings
warnings.filterwarnings('ignore')

plt.style.use('ggplot')

# Overview Data

## Load Data

In [ ]:
# please create a folder named "data"
# all data should be put into "data" folder
file_name = 'green_tripdata_2018-06.csv'
file_path = '../data/'

trips = pd.read_csv(file_path + file_name)

In [ ]:
trips.shape

## Understand Features

In [ ]:
trips.columns.tolist()

* VendorID: LPEP provider that provided the data record (1 = Creative Mobile Technologies, LLC; 2=VeriFone Inc.)
* lpep_pickup_datetime: The time when meter was engaged
* lpep_dropoff_datetime: The time when meter was disengaged
* store_and_fwd_flag: This flag indicates whether the trip record was held in vehicle memory before sending to the vendor, aka “store and forward,” because the vehicle did not have a connection to the server
* RatecodeID: The final rate code in effect at the end of the trip.
* PULocationID: TLC taxi zone for pickup
* DOLocationID: TLC taxi zone for dropoff
* passenger_count: The number of passengers in the vehicle. Recorded by driver
* trip_distance: trip distance in miles
* fare_amount: The time-and-distance fare calculated by the meter
* extra: the 0.50 and 1 rush hour, and overnight charges
* mta_tax: 0.50 MTA tax that is automatically triggered based on the metered rate in use
* tip_amount: This field is automatically populated for credit card tips. **Cash tips are not included**
* tolls_amount: Total amount of all tolls paid in trip
* ehail_fee: NULL
* improvement_surcharge: 0.30 improvement surcharge assessed on hailed trips at the flag drop. The improvement surcharge began being levied in 2015
* total_amount: The total amount charged to passengers. **Does not include cash tips**
* payment_type: how the passenger paid for the trip
* trip_type: A code indicating whether the trip was a street-hail or a dispatch that is automatically assigned based on the metered rate in use but can be altered by the driver

From the above description, we find an interesting thing that might affect the prediction of tips. We do not have cash tips information in the dataset. To make it closer to the Uber situation, we choose *credit card* as the payment type.

In [ ]:
trips_cc = trips[trips['payment_type']==1]

## Shape of Data

In [ ]:
# report rows and columns
print ('Number of samples: ', trips_cc.shape[0])
print ('Number of features: ', trips_cc.shape[1])

# Exploratory Data Analysis

In this section, we derive a new variable, **is_tipped**, which is whether the trip is tipped. In order to get this new variable, we can refer to "tip amount". 

Before creating **is_tipped** variable, we should look at *total amount* and *tip amount* to get some insights.

## Total Amount

In [ ]:
trips_cc['total_amount'].describe()

In [ ]:
# number of samples with total amount less than 3
sum(trips_cc['total_amount']<=3)

For the minimum total amount (extras and surcharges), we believe the records with total amount less than 3 are considered outliers. The quantity of these outliers is small (256 out of 417076), so it is better to remove them.

For the maximum value of 485, it is considered reasonable. 

In [ ]:
trips_cc = trips_cc[trips_cc['total_amount']>3]

In [ ]:
# the distribution of total amount
trips_cc['total_amount'].hist(bins=100)
plt.xlabel("total amount")
plt.ylabel("frequency")
plt.show()

## Tip Amount

In [ ]:
trips_cc['tip_amount'].describe()

The distribution seems good. No need to process tip amount.

## is_tipped

In [ ]:
# create a new feature, named is_tipped
trips_cc['is_tipped'] = trips_cc['tip_amount'].apply(lambda x: 0 if x == 0 else 1)

In [ ]:
# Bar plot
x_pos = ['Tip','No Tip']
x = trips_cc['is_tipped'].value_counts()

plt.bar(x_pos, x, color='green')
plt.ylabel("Counts")
plt.title("Bar plot of whether the trip is tipped")

plt.show()

We can see the data is somehow balanced. Even though, we should still use both accuracy and AUC as evaluation metrics. Besides, we should monitor the running time to see which algorithm runs fast and which is very slow to train.

## Correlations

In [ ]:
correlation = trips_cc.select_dtypes(include = [np.number]).corr()

print(correlation['tip_amount'].sort_values(ascending = False))

We might wonder if tip amount has a high correlation with total amount minus fare amount.

In [ ]:
# remove nan
df = correlation[['tip_amount', 'total_amount', 'fare_amount']].dropna()

# coefficient
np.corrcoef(df['tip_amount'], df['total_amount'] - df['fare_amount'])

Total amount should not be included in the model.

## Boxplot

In [ ]:
# look at relationship between fare_amount and the target has_tip
trips_cc.boxplot(column='fare_amount', by='is_tipped')
plt.ylim(0,100)
plt.ylabel("total amount")
plt.title("")
plt.show()

# Prepare Data For Models

In this section, we prepare data for classification and regression model. First we clean data by removing irrelevant features, checking missing values. Then we use pickup and dropoff time to create some new features that might be useful. Before modeling, we also introduce some feature selection techniques. However, the number of features in this dataset is not too large, so feature selection might not be significantly important. 

## Data Cleaning

In [ ]:
trips_cc.isnull().sum()

The data is actually pretty clean, with no missing value except feature 'ehail_fee', which contains all missing value and should be removed. The dataset is clean because we only choose "credit card" payment type. Some other missing values may occur at other payment type.

Besides, VendorID is id-like variable and should also be removed. Since we already choose credit card, payment_type can be removed.

In [ ]:
# data cleaning

# ehail_fee: drop the column
trips_cc.drop(['ehail_fee', 'VendorID', 'payment_type'],axis=1, inplace=True)

print ('Done with data cleaning!')

## Feature Engineering

### New Features

For Feature Engineering, we create several new features, including day of month, weekday, hour from pickup datetime. We can also generate trip duration and trip speed, which are easy to get from original features. 

In [ ]:
# convert lpep pickup datetime to standard datetime format
trips_cc['pickup_datetime'] = trips_cc['lpep_pickup_datetime'].apply(
    lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
# extract day of month
trips_cc['monthday'] = trips_cc['pickup_datetime'].apply(lambda x: x.day)
# get weekday
trips_cc['weekday'] = trips_cc['pickup_datetime'].apply(lambda x: x.weekday() + 1)
# get hour of day
trips_cc['hour'] = trips_cc['pickup_datetime'].apply(lambda x: x.hour)

# derive Trip duration
trips_cc['dropoff_datetime'] = trips_cc['lpep_dropoff_datetime'].apply(
    lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
trips_cc['trip_duration'] = (trips_cc['dropoff_datetime'] - trips_cc['pickup_datetime']) / np.timedelta64(1, 'm')

# remove duration less than 0
trips_cc = trips_cc[trips_cc['trip_duration'] > 0]

# derive Trip speed with miles per hour
trips_cc['trip_speed'] = trips_cc['trip_distance'] / (trips_cc['trip_duration'] / 60.0)

# remove speed greater than 80
trips_cc = trips_cc[trips_cc['trip_speed'] <= 80]

print ('Done with feature engineering!')

### mta_tax and improvement_surcharge

By analyzing mta_tax and improvement_surcharge, we see the imbalance of the values, indicating that these two features may not contribute much to the predictive models.

In [ ]:
print (trips_cc['mta_tax'].value_counts())

print (trips_cc['improvement_surcharge'].value_counts())

### passenger_count

In [ ]:
print (trips_cc['passenger_count'].value_counts())

* It is impossible to have 0 passenger. To make it simple, remove those samples
* 7/8/9 can be merged to 6.
* Even though it is passenger count, we should regard it as a categorical variable.

In [ ]:
# remove 0 passenger
trips_cc = trips_cc[trips_cc['passenger_count'] != 0]

# merge 7/8/9 to 6 
trips_cc['passenger_count'] = trips_cc['passenger_count'].apply(lambda x: 6 if x >= 6 else x)

### Remove Irrelevant Variables

Datetime-related features are irrelevant. It makes no sense to add those variables to model. In addition, we have already extract some useful information from them. Thus we can remove them.

Store_and_fwd_flag feature indicates whether the trip record was held in vehicle memory before sending to the vendor. This might differ because the vehicle did not have a connection to the server, which has no relationship with tip. We can also remove it.

At last, tip amount and total amount should be removed, since it is response-related variable!

In [ ]:
trips_cc.drop(['lpep_pickup_datetime',
               'lpep_dropoff_datetime',
               'pickup_datetime',
               'dropoff_datetime',
               'store_and_fwd_flag',
               'tip_amount', 'total_amount'],axis=1, inplace=True)


In [ ]:
trips_cc.head()

## Feature Selection 


When there are too many features and it is impossible to handle them manually, one way to solve the problem is to perform feature selection. Even though some models contain this technique (LASSO), which is called embeded feature  seletion, here we introduce filter method using two measure, Chi-square and F-score. 

Please note that, in this dataset, the number of features is not large. Thus, feature selection might not increase the model performance, especially for some complicated models such as Random Forest and Boosting. 

In [ ]:
from sklearn.feature_selection import chi2, f_classif 

def selectFeatures(method, X, y, features):
    """
    Supervised feature selection. Chi2 and Mutual Information are supported

    Args:
        method: str, one of "chi2" and "f"
        X: numpy array, sparse matrix or pandas dataframe
        y: target variable
        features: names of feature to be selected

    Return:
        Sorted list of features by their scores
    """
    # method check
    if method not in ["chi2", "f"]:
        raise Exception("Only Chi2 and f score are supported.")
    if method == "chi2":
        score, _ = chi2(X, y)
    elif method == "f":
        score, _ = f_classif(X, y)
    score = np.nan_to_num(score)
    return sorted(zip(*(features, score)), key=lambda x: x[1], reverse=True)

In [ ]:
features = [x for x in trips_cc.columns if x not in ['is_tip']]
X = trips_cc[features].values
y = trips_cc['is_tipped'].values

chi2 = selectFeatures('chi2', X, y, features)
f = selectFeatures('f', X, y, features)

chi2_df = pd.DataFrame(chi2, columns=['feature','chi2'])
f_df = pd.DataFrame(f, columns=['feature','f_score'])
merged_df = chi2_df.merge(f_df,on='feature',how='inner')

print(merged_df)

In [ ]:
trips_cc.drop(["passenger_count", "trip_speed", "trip_type", "mta_tax", "improvement_surcharge"],
              axis=1, inplace=True)

For both of the feature selection methods, variables "passenger_count", "trip_speed", "trip_type", "mta_tax", "improvement_surcharge" have relatively low scores and I just remove them. 

Now we are ready for the modeling.

# Predictive Modeling

First step of the prediction process is to perform the classification model. The output variable is "is_tipped", with 1 tip and 0 no tip. 

Before applying models, we should divide dataset into two parts, the training set and test set. We use training set to train models, select models and tune hyperparameters. We never use test set until the classification and regression models are completed. 

In [ ]:
# get predictors
predictors = [x for x in trips_cc.columns if x not in ['is_tipped']]

# train test split
X_train, X_test, y_train, y_test = train_test_split(trips_cc[predictors].values, trips_cc['is_tipped'].values, 
                                              test_size=0.2, random_state=42)

## Model Selection

* Roughly train Logistic Regression, Decision Tree, Bagging, SVM, Random Forest, AdaBoost and GBDT
* Choose a best model to tune hyperparameters using cross validation. If multiple models have similar performance, tune both

In [ ]:
# model selection using cross validation 
def cross_validation(X, y, cv, classifier,**kwargs):
    """
    @input
        X:          feature space 
        y:          label
        classifier: classification model
        cv:         num of folds   
        **kwargs:   parameters in classifier
    @output:
        y_predict
    """ 
    start = time.time()
    kf = KFold(n_splits=cv,shuffle=True)
    y_predict = y.copy()
    clf = classifier(**kwargs)
    
    accuracy = 0
    auc = 0
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf.fit(X_train,y_train)
        y_predict = clf.predict(X_test)
        accuracy += np.mean(y_predict==y_test)
        auc += roc_auc_score(y_test, y_predict)
    
    end = time.time()
    return accuracy/cv, auc/cv, end-start

In [ ]:
# run classification model

# potential models
models = [LogisticRegression, DecisionTreeClassifier, 
          BaggingClassifier, RandomForestClassifier, 
          AdaBoostClassifier, GradientBoostingClassifier]

# results
res_dict = {}

# train models
for model in models:
    print ("Training model: ", model)
    tmp_acc, tmp_auc, tmp_runtime = cross_validation(X_train, y_train, 5, model)
    res_dict[model] = [tmp_acc, tmp_auc, tmp_runtime]

print ('Done with classification model selection!')

In [ ]:
res_df = pd.DataFrame(res_dict).T
res_df.columns = ["Accuracy","AUC","RunTime"]

res_df

The performance result is from the 5-fold Cross Validation. Even though Bagging performed the best, I would like to choose **Random Forest** as my classification model for two reasons.

* Random Forest is much faster.
* Random Forest supports feature importance.

Then we could use RandomSearch to automatically tune hyperparameters.

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html

## Tune Random Forest

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In sklearn, there are several hyperparameters to tune in Random Forest.

* n_estimators: The number of base estimators in the ensemble
* max_depth: The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.
* min_samples_split: The minimum number of samples required to split an internal node

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

RF_params = {
    'n_estimators': [10,50,100],
    'max_depth': [5,20,None],
    'min_samples_split': [2,5,10]
                 }
start = time.time()

rs_RF = RandomizedSearchCV(
    RandomForestClassifier(), # classifier
    RF_params, # your parameter distribution
    n_iter = 20, # number of candidates among total that you will randomly search for
    scoring = "roc_auc", # evaluation metrics
    cv=5, # number of folds
    verbose=True)

rs_RF.fit(X_train, y_train)

print ("RandomizedSearchCV took %.2f seconds for Random Forest"%(time.time() - start))

In [ ]:
# print the results
rs_RF.cv_results_

In [ ]:
# get the optimal parameters
rs_RF.best_params_

In [ ]:
# best AUC 
rs_RF.best_score_

## Train Random Forest

In [ ]:
# use the best parameters to train
RF_best = RandomForestClassifier(n_estimators=100, min_samples_split=5, max_depth=None)
RF_best.fit(X_train, y_train)
y_pred_train = RF_best.predict(X_train)
y_pred = RF_best.predict(X_test)

print ("AUC on training set: ", roc_auc_score(y_train, y_pred_train))
print ("AUC on test set: ", roc_auc_score(y_test, y_pred))

## Feature Importance

In [ ]:
# show feature importances
feature_importance = pd.DataFrame(predictors, columns=['feature'])
feature_importance['importance'] = RF_best.feature_importances_
feature_importance.plot(x='feature',y='importance',xticks=range(len(predictors)),rot=60,figsize=(8,4))
plt.ylabel('feature importance')
plt.title("Figure 14: The importance values of features")
plt.show()

# Conclusion

We use 7 classification models to fit green taxi data, in order to classify if a trip will be tipped or not. During model selection, Bagging, Random Forest and Gradient Boosting performed better, which is reasonable. Logistic Regression or Decision Tree seems too simple for this data, and somehow underfitting. 

Even though Bagging performed the best, we choose Random Forest as the final model for two reasons. First, Random Forest run twice as fast as Bagging. Second, feature importance can be generated in Random Forest, while Bagging does not support this function. 

By tunning Random Forest, we get **{'n_estimators': 100, 'min_samples_split': 5, 'max_depth': None}** as the best parameters, using Random Search Cross Validation. We then train the whole training set using these hyperparameters and test on test set. 

* AUC 0.96 on training set
* AUC 0.76 on test set

The result shows overfitting. Even though we tried several parameters that might lead to underfitting, it still shows overfitting. The problem may come from the dataset. 

* Internally, the split might differ a lot
* Externally, we can set a higher proportion of test set

In the end, we analyze the feature importance. Trip duration/distance and pickup/dropoff location stand out. This result makes sense. First, long trip might have more chance to be tipped, since passengers will feel that the driver is working hard and give some tips. Second, certain locations indicate certain kind of people, who are more likely to give tips. 

By using this information, drivers who are likely to be tipped can get the order with long distance, but it may take a longer time. Also, Uber or some other companies can use location information to get more drivers among those area. 

